In [296]:
import requests
import json
import pprint


URL = "https://csgolounge.com"

session = requests.Session()

# Make the user agent look like a browser
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:24.0) Gecko/20140624 Firefox/24.0 Iceweasel/24.7.0'
})

In [297]:
response = session.get(URL)

print(f"status code: {response.status_code} {response.reason}") 

status code: 200 OK


In [298]:
print(response.text)


<!DOCTYPE html>
<html lang="en">
<head>
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <meta lang="en" />
    <meta name="google" content="notranslate" />
    <title>CSGOLOUNGE.COM - CSGO and eSports betting exchange!</title>

            <link rel="shortcut icon" type="image/x-icon" href="/images/lounge/favicon_csgo.ico">
        <meta property="og:image" content="/images/lounge/csgo/og.png"/>
    
    <meta property="og:title" content="CSGOLOUNGE.COM - CSGO and eSports betting exchange!" />
    <meta name="description" property="og:description" content="Bet on CS:GO for the best odds, we have all the top matches! Fast technical support and instant withdrawal." />

    <meta name="apple-mobile-web-app-title" content="Bets.net">

        <link rel="alternate" href="https://csgolounge.com/cz/" hreflang="cs" /><link rel="alternate" href="https://csgolounge.com/de/" hreflang="de" /><

In [299]:
# Extract the JSON data from the function call in the HTML
# The line looks like this:
# this.populateBets([{...}, {...}, ..., {...}]);

# Find the start of the JSON data
start = response.text.find("this.populateBets(")
start += len("this.populateBets(")

# Find the end of the JSON data
end = response.text.find(");", start)

# Extract the JSON data
json_data = response.text[start:end]

# Parse the JSON data
lounge_data = json.loads(json_data)

# Pretty print the data
# pprint.pprint(lounge_data[0])

In [300]:
# Data processing

# Sort the match data by the time the match starts
lounge_data.sort(key=lambda x: x["m_time"])

# m_status:
# 0: Upcoming
# 1: Live
# 2: Finished (Team 1 won)
# 3: Finished (Team 2 won)
# 4: Unknown
# 5: Cancelled
# 6: Rescheduled
# 7: Draw

upcoming = [lounge_data[i] for i in range(len(lounge_data)) if lounge_data[i]["m_status"] == "0"]

In [301]:
# We must unify the names of the teams between CSGOLounge and Bovada

# Bovada : CSGOLounge
# -------------------
# Astralis : Astralis
# BIG : BIG
# G2 : G2
# Team Liquid : Liquid
# Cloud9 : Cloud9
# Rebels Gaming : Rebels
# Eternal Fire : Eternal Fire
# BB Team : BB Team
# Ence : Ence
# Heroic : Heroic
# FURIA Esports : FURIA
# TheMongolz : TheMongolz
# Team Spirit : Spirit
# Apeks : Apeks
# GamerLegion : GamerLegion
# M80 : M80
# Rooster : Rooster
# Virtus.pro : VP
# Alliance : Alliance
# Spirit Academy : Spirit Academy
# Sangal Esports : Sangal
# ECSTATIC : ECSTATIC
# Into The Breach : Into the Breach
# BIG OMEN Academy : BIG Academy
# Bleed : BLEED
# Metizport : Metizport
# Preasy : Preasy
# Entropiq : Entropiq
# Sprout : Sprout
# Alternate Attax : ALTERNATE aTTaX

lounge_to_bovada = {
    "Astralis": "Astralis",
    "BIG": "BIG",
    "G2": "G2",
    "Liquid": "Team Liquid",
    "Cloud9": "Cloud9",
    "Rebels": "Rebels Gaming",
    "Eternal Fire": "Eternal Fire",
    "BB": "BB Team",
    "Ence": "Ence",
    "Heroic": "Heroic",
    "FURIA": "FURIA Esports",
    "TheMongolz": "TheMongolz",
    "Spirit": "Team Spirit",
    "Apeks": "Apeks",
    "GamerLegion": "GamerLegion",
    "M80": "M80",
    "Rooster": "Rooster",
    "VP": "Virtus.pro",
    "Alliance": "Alliance",
    "Spirit Academy": "Spirit Academy",
    "Sangal": "Sangal Esports",
    "ECSTATIC": "ECSTATIC",
    "Into the Breach": "Into The Breach",
    "BIG Academy": "BIG OMEN Academy",
    "BLEED": "Bleed",
    "Metizport": "Metizport",
    "Preasy": "Preasy",
    "Entropiq": "Entropiq",
    "Sprout": "Sprout",
    "ALTERNATE aTTaX": "Alternate Attax"
}

In [302]:
from dataclasses import dataclass

@dataclass
class LoungeMatch:
    time: int
    status: int
    team1: str
    team2: str
    t1_value: float
    t2_value: float

    @property
    def total_value(self):
        return self.t1_value + self.t2_value
    
    @property
    def t1_odds(self):
        return self.t1_value / self.total_value if self.total_value != 0 else 0
    
    @property
    def t2_odds(self):
        return self.t2_value / self.total_value if self.total_value != 0 else 0
    
    @property
    def t1_multiplier(self):
        return 1 / self.t1_odds if self.t1_odds != 0 else 0
    
    @property
    def t2_multiplier(self):
        return 1 / self.t2_odds if self.t2_odds != 0 else 0
    
    @staticmethod
    def from_dict(match_dict):
        time = int(match_dict["m_time"])
        status = int(match_dict["m_status"])

        # We arbitrarily choose to unify the names of the teams to the Bovada names
        team1 = lounge_to_bovada[match_dict["t1name"]] if match_dict["t1name"] in lounge_to_bovada else match_dict["t1name"]
        team2 = lounge_to_bovada[match_dict["t2name"]] if match_dict["t2name"] in lounge_to_bovada else match_dict["t2name"]

        # If no bets have been placed on the match, the sumbets key will not exist
        if "sumbets" not in match_dict:
            return LoungeMatch(time, status, team1, team2, 0, 0)
        
        # Convert the currencies to USD and calculate the total value of each team

        TO_USD = {"EUR": 1.09, "RUB": 0.011, "USD": 1}
        sumbets = match_dict["sumbets"]

        t1_value = 0
        t2_value = 0

        for currency in sumbets:
            if "1" not in sumbets[currency]:
                sumbets[currency]["1"] = 0

            if "2" not in sumbets[currency]:
                sumbets[currency]["2"] = 0

            t1_value += sumbets[currency]["1"] * TO_USD[currency]
            t2_value += sumbets[currency]["2"] * TO_USD[currency]

        return LoungeMatch(time, status, team1, team2, t1_value / 100, t2_value / 100)
    
lounge_matches = []

for match in lounge_data:
    lounge_match = LoungeMatch.from_dict(match)
    # print(f"{lounge_match.team1} vs {lounge_match.team2} @ {lounge_match.time}")
    # print(f"    ${lounge_match.t1_value:0.2f} vs ${lounge_match.t2_value:0.2f}")
    # print(f"    {lounge_match.t1_odds * 100:0.2f}% vs {lounge_match.t2_odds * 100:0.2f}%")
    # print(f"    {lounge_match.t1_multiplier:0.2f}x vs {lounge_match.t2_multiplier:0.2f}")
    # print()
    lounge_matches.append(lounge_match)
        

In [303]:
# Convert moneyline odds to implied probability

def moneyline_to_probability(moneyline):
    if moneyline > 0:
        return 100 / (moneyline + 100)
    else:
        return -moneyline / (-moneyline + 100)


In [304]:
# Get Bovada.lv odds

URL = "https://www.bovada.lv/services/sports/event/coupon/events/A/description/esports/counter-strike-2"
# URL = "https://www.bovada.lv"
response = session.get(URL)

print(f"status code: {response.status_code} {response.reason}")

status code: 200 OK


In [305]:
data = response.json()

events = []

for obj in data:
    events.extend(obj["events"])

# pprint.pprint(events[0])

In [306]:
from dataclasses import dataclass

@dataclass
class BovadaMatch:
    time: int
    team1: str
    team2: str
    team1_moneyline: float
    team2_moneyline: float

    @property
    def t1_odds(self):
        return moneyline_to_probability(self.team1_moneyline) / (moneyline_to_probability(self.team1_moneyline) + moneyline_to_probability(self.team2_moneyline))
    
    @property
    def t2_odds(self):
        return moneyline_to_probability(self.team2_moneyline) / (moneyline_to_probability(self.team1_moneyline) + moneyline_to_probability(self.team2_moneyline))
    
    @property
    def t1_multiplier(self):
        return 1 / self.t1_odds if self.t1_odds != 0 else 0
    
    @property
    def t2_multiplier(self):
        return 1 / self.t2_odds if self.t2_odds != 0 else 0
    
    def reversed(self):
        return BovadaMatch(
            time=self.time,
            team1=self.team2,
            team2=self.team1,
            team1_moneyline=self.team2_moneyline,
            team2_moneyline=self.team1_moneyline
        )
    
    @staticmethod
    def from_event(event):
        return BovadaMatch(
            time=int(event["startTime"] / 1000),
            team1=event["competitors"][0]["name"],
            team2=event["competitors"][1]["name"],
            team1_moneyline=float(event["displayGroups"][0]["markets"][0]["outcomes"][0]["price"]["american"]),
            team2_moneyline=float(event["displayGroups"][0]["markets"][0]["outcomes"][1]["price"]["american"])
        )

bovada_matches = []

for event in events:
    if len(event["competitors"]) != 2:
        continue    

    match = BovadaMatch.from_event(event)
    # print(match)
    bovada_matches.append(match)


In [307]:
pairs = []

for bm in bovada_matches:
    # Find the corresponding LoungeMatch
    least_diff = 1e9
    closest_match = None
    for lm in lounge_matches:
        diff = abs(lm.time - bm.time)
        if diff < least_diff and lm.team1 == bm.team1 and lm.team2 == bm.team2:
            least_diff = diff
            closest_match = lm

    # If no match was found, try reversing the teams
    if closest_match is None:
        
        bm = bm.reversed()
        for lm in lounge_matches:
            diff = abs(lm.time - bm.time)
            if diff < least_diff and lm.team1 == bm.team1 and lm.team2 == bm.team2:
                least_diff = diff
                closest_match = lm
    
    # If no match was found after reversing the teams, output that no match was found
    if closest_match is None:
        # print(f"(Bovada) {bm.team1} vs {bm.team2} @ {bm.time}")
        # print(f"    No match found")
        # print()
        continue

    # print(f"(Bovada) {bm.team1} vs {bm.team2} @ {bm.time}")
    # print(f"(Lounge) {closest_match.team1} vs {closest_match.team2} @ {closest_match.time}")
    # print()

    pairs.append((bm, closest_match))




In [308]:
# Calculate the expected value of each bet

def expected_value(bm, lm):
    # We take the Bovada odds as the true odds, and the Lounge multiplier as the bet winnings
    # +1 EV -> We expect to win $1 for every $1 we bet
    #  0 EV -> We expect to break even
    # -1 EV -> We expect to lose $1 for every $1 we bet
    
    t1_ev = bm.t1_odds * lm.t1_multiplier - 1
    t2_ev = bm.t2_odds * lm.t2_multiplier - 1

    return t1_ev, t2_ev

print(len(pairs))
for bm, lm in pairs:
    t1_ev, t2_ev = expected_value(bm, lm)

    print(f"{lm.team1} vs {lm.team2} @ {lm.time}")
    print(f"    Existing value: ${lm.t1_value:0.2f} vs ${lm.t2_value:0.2f}")
    print(f"    (Lounge) {lm.t1_odds * 100:0.2f}% vs {lm.t2_odds * 100:0.2f}%")
    print(f"    (Bovada) {bm.t1_odds * 100:0.2f}% vs {bm.t2_odds * 100:0.2f}%")
    print(f"    Payout: {lm.t1_multiplier:0.2f}x vs {lm.t2_multiplier:0.2f}x")
    print(f"    Expected Value: ${t1_ev:0.2f} vs ${t2_ev:0.2f}")
    print()

14
Astralis vs BIG @ 1706448656
    Existing value: $1523.49 vs $56.27
    (Lounge) 96.44% vs 3.56%
    (Bovada) 70.84% vs 29.16%
    Payout: 1.04x vs 28.08x
    Expected Value: $-0.27 vs $7.19

G2 vs Team Liquid @ 1706457656
    Existing value: $39.61 vs $133.63
    (Lounge) 22.86% vs 77.14%
    (Bovada) 57.21% vs 42.79%
    Payout: 4.37x vs 1.30x
    Expected Value: $1.50 vs $-0.45

Cloud9 vs Rebels Gaming @ 1706711404
    Existing value: $2.56 vs $0.74
    (Lounge) 77.67% vs 22.33%
    (Bovada) 81.10% vs 18.90%
    Payout: 1.29x vs 4.48x
    Expected Value: $0.04 vs $-0.15

Eternal Fire vs BB Team @ 1706711404
    Existing value: $15.75 vs $0.00
    (Lounge) 100.00% vs 0.00%
    (Bovada) 54.16% vs 45.84%
    Payout: 1.00x vs 0.00x
    Expected Value: $-0.46 vs $-1.00

Ence vs BIG @ 1706715004
    Existing value: $1.07 vs $0.28
    (Lounge) 78.94% vs 21.06%
    (Bovada) 43.67% vs 56.33%
    Payout: 1.27x vs 4.75x
    Expected Value: $-0.45 vs $1.67

Heroic vs Astralis @ 1706715004
  

In [309]:
side1_moneyline = -280
side2_moneyline = 205
    
side1_probability = moneyline_to_probability(side1_moneyline)
side2_probability = moneyline_to_probability(side2_moneyline)

side1_payout = 1 / side1_probability
side2_payout = 1 / side2_probability

total_probability = side1_probability + side2_probability

print(f"Side 1: {side1_probability / total_probability * 100:0.2f}% Fair payout: {side1_payout:0.2f}x")
print(f"Side 2: {side2_probability / total_probability * 100:0.2f}% Fair payout: {side2_payout:0.2f}x")
print(f"House edge: {(total_probability - 1) * 100:0.2f}%")

Side 1: 69.21% Fair payout: 1.36x
Side 2: 30.79% Fair payout: 3.05x
House edge: 6.47%
